In [18]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os
from io import StringIO
import glob
import re

In [2]:
crew_dict = {"Scarify":["Scarify"], "Strip Site":["Strip Site"], 
             "Soil Correction":["Soil Correction", "OX","OE", "Over Ex", "Excavate & Re-compact", "Over-Ex", "Excavate & Recompact"],
             "Ex Retainining Wall":["Excavate Site Retaining", "Excavate MSE", "Exc Retaining", "Exc Site Retn Walls"],
             "Export":["Export", "Exp"], "Import":["Import", "Imp"],
             "Ex Frost footing": ["Exc Fro", "Excavate Fro", "Footing Excavation"], 
             "Ex Trash Enc":["Excavate Trash", "Exc Trash"],
             "Cut to Fill":["Cut To Fill", "Utility Spoils - Site Fill", "Building Pad Fill w/ Spoils"],
             "Cut to Stockpile":["Cut to Stockpile", "Load & Stockpile"],
             "Handle spoils":["Handle Utility Spoils", "Handle Foundation Spoils", "Handle Spoils", "Handle & Stockpile"],
             "Double Handle":["Double Handle"], "Cut in C&G":["Cut In C", "Cut-in C", "Grade C&G"],
             "Ex Elevator":["Excavate Elevator"],
             "Grade Landscape":["grade Landscape", "grade Green", "Landscape"],
             "Grade Parkinglots and Roads":["Grade Boulevards","Grade Asphalt", "Grade Parking", "Grade Pkg", "Grade Main Roads"],
             "Grade SW":["Grade Sidewalks", "Grade Walks", "Exterior Slabs", "grade Brick Pavers", "Grade SW"],
             "Export":["Export"],
             "Grade BLDG": ["Grade Bldg", "Grade Building", "Grade Structural"],
             "Drain Tile":["Drain"],
             "Backfill Int":["Backfill Int", "Footing Backfill"],
             "Backfill Ext":["Backfill Ext"],
             "Backfill Elevator":["Backfill Elevator"],
             "Backfill Trash":["Backfill Trash"], "Backfill Frost": ["Backfill Frost"],
             "High Backfill": ["High Backfill"], "Grade Class X":["Grade Class"],
             "Excavate Interior":["Excavate Int", "Excavate Thickened", "Excavate Grade Beams", "Exc Int"],
             "Excavate Exterior":["Excavate Ext", "Exc Ext"], "Grade Pond":["Basin Grading","Grade Pond" , "Grade Detention Pond"],
             "Assist shoring":["Assist Shoring"],
             "Stairs":["Stairs"], "Place Strippings":["Place Stripping"],
             "Street Sweep":["Street Sweeping"]}

In [142]:
completed_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections"
mk2_recents = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\mk2_compiled_data.xlsx"
ew = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\Earth_Work.xlsx"

on_going_path = ''

In [126]:
def job_date(df, date):
    job = str(df.columns[0]).split(' ', 1)[1]
    edit_date = pd.to_datetime(date, format='%Y%m%d', errors='coerce')
#     attendance = str(df.iloc[0,0]).split(' ',1)[1]
    return job, edit_date

def job_code(completed_path):
    files = {}
    for item in os.listdir(completed_path):
        job_code = item.split(' ', 2)[1]
        job_date = item.split(' ', 2)[-1]
        date = re.sub("[^\d\.]", "", job_date)
        date = date.replace(".", "")
        j_dict = {job_code:date[-8:]}
        for key in j_dict:
            if key not in files or j_dict[key] > files[key]:
                files[key] = j_dict[key]

#         files.append({"job_code": job_code, "date": date})
#         files.append(j_dict)
#     files= pd.unique(files).tolist()
    return files
    
    
def lat_old(files):
    lat_file_list = []
    for job, date in files.items():
        job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if job and date in x]
#         latest_file = max(job_files, key=os.path.getctime)
        lat_file_list.append((job_files[0], date, job))
#     lat_file_list = pd.unique(lat_file_list)
    return lat_file_list

def files_to_dfs(file_list):
    df_list = []
#     for item in file_list:
    for item in file_list:
        num_sheet = len(pd.ExcelFile(item[0]).sheet_names)
        for sheet in range(0,num_sheet):
            xl = pd.read_excel(item[0] ,sheet_name=sheet)
            df = pd.DataFrame(xl)
            job, date = job_date(df, item[1])
            df.columns = df.iloc[2]
            df = df.drop(df.index[:3])
            df = df.iloc[:, :27]
            df['updated'] = date
            df['job'] = job
            df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
            df = df.dropna(subset=['Qty'])
            df = df.reset_index(drop=True)
            df_list.append(df)
    full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
    return full_df

In [146]:
job_codes = job_code(completed_path)
newest = lat_old(job_codes)
full_df = files_to_dfs(newest)

In [147]:
full_df[full_df['job'].isnull()]

2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Eq,Mtl Comm Costs,Reclass,Total New Costs,NaN,VP + HJ Cost to Date,NaN,New Projected Budget,updated,job


In [139]:
full_df['Ph Int'] = full_df['Phase Code'].astype(str).str[0]
full_df = full_df[~full_df['updated'].isnull()]
full_df['year'] = pd.DatetimeIndex(full_df['updated']).year
full_df['month'] = pd.DatetimeIndex(full_df['updated']).month

full_df = full_df.sort_values(by='updated', ascending=False)

no_dup = full_df.drop_duplicates(subset=['Phase Code','Description','job'], keep='last')
no_dup['Qty'] = no_dup['Qty'].astype(float)
no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
no_dup['Total Budget'] = no_dup['Total Budget'].astype(float)

most_recent = no_dup.sort_values(by=['job', 'Phase Code'])
most_recent = most_recent[~most_recent[['Qty','HJ Qty Comp']].eq(0).sum(1).ge(2)]

earth_work = most_recent[most_recent['Ph Int'] == '3']

earth_work['crew'] = 'misc.'

for key, values in crew_dict.items():
    earth_work.loc[earth_work['Description'].str.lower().str.contains('|'.join([x.lower() for x in values])), "crew"] = key

reduced = earth_work.loc[:, earth_work.columns.notna()]
reduced = reduced.drop(['% Comp', 'Notes','Lab','Eq', 'Reclass'], axis=1)

C:\Users\willd\AppData\Local\Temp\ipykernel_14752\4185029306.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Qty'] = no_dup['Qty'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_14752\4185029306.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['Actual Cost$'] = no_dup['Actual Cost$'].astype(float)
C:\Users\willd\AppData\Local\Temp\ipykernel_14752\4185029306.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [140]:
test = reduced
test.loc[test['Total Budget'] > 0, 'Est UC'] = round(test['Total Budget'] / test['Qty'], 2)
test.loc[test['Actual Cost$'] > 0, 'Act. UC'] = round(test['Actual Cost$'] / test['HJ Qty Comp'], 2)

In [174]:
for j in newest:
    print(j[1:3])

('20220629', '10291-30')
('20220613', '10291-30-50')
('20220601', '10291-50')
('20220627', '10292-50')
('20220616', '10293-50')
('20220321', '10298-30')
('20220810', '10298-30-50')
('20220627', '10298-50')
('20220505', '10299-50')
('20220207', '10301')
('20220112', '10304')
('20220629', '10304-30')
('20220613', '10304-30-50')
('20220126', '10304-50')
('20220406', '10305-30')
('20230613', '10305-50')
('20230116', '10308')
('20220309', '10308-30')
('20230510', '10308-30-50')
('20230607', '10308-50')
('20220919', '10310-30')
('20230116', '10310-30-50')
('20221128', '10310-50')
('20221018', '10312-50')
('20230116', '10314-30-50')
('20230116', '10314-50')
('20220705', '10315-30-50')
('20230116', '10315-50')
('20220616', '10316-30-50')
('20220627', '10316-50')
('20220616', '10317-30')
('20220616', '10318-30-50')
('20220728', '10320-30')
('20230222', '10320-30-50')
('20221024', '10320-50')
('20220801', '10321-30-50')
('20220829', '10321-50')
('20220705', '10325-30')
('20220505', '10329')
('20

In [143]:
writer = pd.ExcelWriter(mk2_recents, engine="xlsxwriter")
most_recent.to_excel(writer, sheet_name="mk1")
test.to_excel(writer, sheet_name="agg data")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()

In [148]:
file = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\compiled_data.xlsx"
xl = pd.read_excel(file ,sheet_name=1)
df = pd.DataFrame(xl)

In [159]:
df_all = df.merge(test.drop_duplicates(), on=['Description', 'job'], 
                   how='left', indicator=True)

In [161]:
df_all[df_all[]=='left_only']

,Unnamed: 0.1,Unnamed: 0,Phase Code_x,Description,Qty_x,HJ Qty Comp_x,Unit_x,% Comp,Budget Last Projection_x,New Budgets_x,...,VP + HJ Cost to Date_y,New Projected Budget_y,updated_y,Ph Int_y,year_y,month_y,crew_y,Est UC_y,Act. UC_y,_merge
60,60,645,3005,Strip Site,265.0,265,CY,1.000000,7937.48,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
61,61,646,3010,Cut to Fill,518.0,518,CY,1.000000,6808.66,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
62,62,647,3015,Soil Correction,6884.0,6884,BCY,1.000000,38938.83,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
63,63,648,3020,Load & Export,2030.0,2931,BCY,1.443842,41987.56,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
64,64,649,3025,Handle & Stockpile Geo Pier Spoils,447.0,1882,BCY,4.210291,25749.63,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,472,3709,3143,PR-03 Walk Grade Sidewalks,1301.0,1301,SF,1.000000,1375.88,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
473,473,3710,3405,"Remove 18"" to TOP under Structural Slab",596.0,891,BCY,1.494966,14466.42,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
474,474,3711,3406,"Backfill 18"" to TOP under Structural Slab",596.0,596,BCY,1.000000,16704.67,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
475,475,3712,3407,Backfill Frost Footings One Side,675.0,675,LF,1.000000,5525.53,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [160]:
df_all

,Unnamed: 0.1,Unnamed: 0,Phase Code_x,Description,Qty_x,HJ Qty Comp_x,Unit_x,% Comp,Budget Last Projection_x,New Budgets_x,...,VP + HJ Cost to Date_y,New Projected Budget_y,updated_y,Ph Int_y,year_y,month_y,crew_y,Est UC_y,Act. UC_y,_merge
0,0,14,3030,Import & Handle Class 6,465.0,2823,TON,6.070968,8458.25,NaN,...,46214.57,46214.57,2022-01-12,3,2022.0,1.0,Import,18.19,16.37,both
1,1,15,3040,Fine Grade Class 6,0.0,38234,SF,NaN,2991.39,NaN,...,20870.55,20870.55,2022-01-12,3,2022.0,1.0,Grade Class X,inf,0.55,both
2,2,16,3100,Site Grading Cut to Fill,2489.0,1444,BCY,0.580153,10564.06,NaN,...,8459.39,8459.39,2022-01-12,3,2022.0,1.0,Cut to Fill,4.24,5.86,both
3,3,17,3200,Utility Spoils - Site Fill,1623.0,0,BCY,0.000000,7295.30,NaN,...,0,0,2022-01-12,3,2022.0,1.0,Cut to Fill,4.49,NaN,both
4,4,18,3300,Excavate Frost Footings/Grade Beams/Inline Pie...,2223.0,2223,LF,1.000000,20179.80,NaN,...,16684.65,16684.65,2022-01-12,3,2022.0,1.0,Ex Frost footing,9.08,7.51,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,472,3709,3143,PR-03 Walk Grade Sidewalks,1301.0,1301,SF,1.000000,1375.88,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
473,473,3710,3405,"Remove 18"" to TOP under Structural Slab",596.0,891,BCY,1.494966,14466.42,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
474,474,3711,3406,"Backfill 18"" to TOP under Structural Slab",596.0,596,BCY,1.000000,16704.67,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
475,475,3712,3407,Backfill Frost Footings One Side,675.0,675,LF,1.000000,5525.53,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [164]:
test[test['updated'].isnull()]

2,Phase Code,Description,Qty,HJ Qty Comp,Unit,Budget Last Projection,New Budgets,Total Budget,Actual Cost$,Cmtd Cost$,...,VP + HJ Cost to Date,New Projected Budget,updated,job,Ph Int,year,month,crew,Est UC,Act. UC


In [ ]:
len(full_df)

In [145]:
len(full_df)

3667

In [129]:
full_df[full_df['job'].isnull()]

2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Eq,Mtl Comm Costs,Reclass,Total New Costs,NaN,VP + HJ Cost to Date,NaN,New Projected Budget,updated,job


In [124]:
len(newest)

66

In [125]:
newest

[('C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10291-30 Stockyard SYEC NWC - A&P 20220629.xlsx',
  '20220629',
  '10291-30'),
 ('C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10291-30-50 Stockyard SYEC NWC - A&P 20220613.xlsx',
  '20220613',
  '10291-30-50'),
 ('C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10291-50 Stockyard SYEC NWC - A&P 20220601.xlsx',
  '20220601',
  '10291-50'),
 ('C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10292-50 Sorrel Ranch-Lokal 20220627.xlsx',
  '20220627',
  '10292-50'),
 ('C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10292-50 Sorrel Ranch-Lokal 20220616.xlsx',
  '20220616',
  '10293-50'),
 ('C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\co

In [67]:
unique_jc = [dict(t) for t in {tuple(d.items()) for d in job_codes}]
print(len(job_codes), len(unique_jc))

407 401


In [144]:
print(len(newest))

66


In [136]:
ew_ = [x for x in job_code(completed_path) if '-30' in x]
unique_cod = sorted(pd.unique([x[0] for x in (x.split("-") for x in ew_)]))
ew_code = [x for x in full_df['job'] if '-30' in x]
unique_codes = sorted(pd.unique([x[0] for x in (x.split("-") for x in ew_code)]))
print(len(unique_codes))
print(len(unique_cod))

11
26


In [133]:
ew_ = [x for x in job_code(completed_path) if '-30' in x]
unique_cod = sorted(pd.unique([x[0] for x in (x.split("-") for x in ew_)]))

In [130]:
ful = files_to_dfs(newest)
ew_code = [x for x in ful['job'] if '-30' in x]
unique_codes = sorted(pd.unique([x[0] for x in (x.split("-") for x in ew_code)]))

In [134]:
print(len(unique_codes))
print(len(unique_cod))

11
26


In [137]:
full_df

2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Eq,Mtl Comm Costs,Reclass,Total New Costs,NaN,VP + HJ Cost to Date,NaN,New Projected Budget,updated,job
0,1001,Contingency,NaN,1,0,LS,0,NaN,7020.74,NaN,...,NaN,NaN,NaN,0,NaN,5891.04,NaN,5891.04,2022-06-29,10291-30 Stockyard Event Center NWC-A&P
1,1010,Field Supervision,NaN,18,15,WKS,0.833333,NaN,9892.29,NaN,...,NaN,NaN,NaN,0,NaN,12106.27,NaN,12106.27,2022-06-29,10291-30 Stockyard Event Center NWC-A&P
2,1020,PM/Admin,NaN,18,0,WKS,0,NaN,28092.45,NaN,...,NaN,NaN,NaN,0,NaN,38712.75,NaN,38712.75,2022-06-29,10291-30 Stockyard Event Center NWC-A&P
3,1030,Safety/QC,NaN,1,0,LS,0,NaN,5411.19,NaN,...,NaN,NaN,NaN,0,NaN,2889,NaN,2889,2022-06-29,10291-30 Stockyard Event Center NWC-A&P
4,1040,Unproductive Equipment Hours,NaN,0,0,HRS,NaN,NaN,0,NaN,...,NaN,NaN,NaN,0,NaN,25414.82,NaN,25414.82,2022-06-29,10291-30 Stockyard Event Center NWC-A&P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662,7091,Concrete,NaN,18,0,CY,0,NaN,4531.4,NaN,...,NaN,NaN,NaN,0,NaN,0,NaN,0,2023-11-28,10378 Brunner Farm-CGRS
3663,7092,Rebar,NaN,2,0,EA,0,NaN,1813.88,NaN,...,NaN,907.01,NaN,907.01,NaN,907.01,NaN,907.01,2023-11-28,10378 Brunner Farm-CGRS
3664,7099,Miscellaneous,NaN,1,0,LS,0,NaN,829.2,NaN,...,NaN,NaN,NaN,0,NaN,0,NaN,0,2023-11-28,10378 Brunner Farm-CGRS
3665,7105,Generator Pad Foundation,NaN,1,0,EA,0,NaN,1929.32,NaN,...,NaN,NaN,NaN,0,NaN,1884.58,NaN,1884.58,2023-11-28,10378 Brunner Farm-CGRS
